# **Report 5 Summary**

1. Download representitive sequences from Gencode gencode_repr.fasta
2. Perform exact alignment with all the SRR11015356 reads against the gencode_repr.fasta 
3. Perform exact alignment with the before75_after75 unitigs file against gencode_repr.fasta
4. Generate coverage graphs using the two generated bam files from the previous steps


## **1.1 Data Download**

```bash
mkdir -p data && cd data/

#1 Human Transcriptome
wget -c ftp://ftp.ebi.ac.uk/pub/databases/gencode/Gencode_human/release_33/gencode.v33.transcripts.fa.gz

#2 Basic gene annotation CHR GTF
wget -c ftp://ftp.ebi.ac.uk/pub/databases/gencode/Gencode_human/release_33/gencode.v33.basic.annotation.gtf.gz
```

## **1.2 Representitive sequences extractions**

```bash
python extract_transcripts_from_gtf.py data/gencode.v33.basic.annotation.gtf data/gencode.v33.transcripts.fa
```

## 2. Aligning the before75_after75 on the reprs. transcripts **(DIAMOND)**

### 2.1 Creating index
```bash
diamond makedb --in extractedGTF_gencode.v33.transcripts.fa --db extractedGTF_gencode.v33.transcripts
## This creates extractedGTF_gencode.v33.transcripts.dmnd
```

### 2.2 Query
```bash
export DATA=/home/mabuelanin/Desktop/kexpression_experiment/symbolic/data/drtamer_data

```

```bash
bwa index extractedGTF_gencode.v33.transcripts.fa
bwa mem extractedGTF_gencode.v33.transcripts.fa $DATA/reps_unitigs_SRR11015356_beforek75_afterk75.fa.unitigs.fa -t 4 > bwa_mem_aln.sam
samtools view -S -b bwa_mem_aln.sam -o bwa_mem_aln.bam
samtools sort bwa_mem_aln.bam -o sorted_bwa_mem_aln.bam
samtools index sorted_bwa_mem_aln.bam
samtools depth sorted_bwa_mem_aln.bam > sorted_bwa_mem_aln.bam.cov
cat sorted_bwa_mem_aln.bam.cov | awk -F'\t' '{print $3}' | sort -n | uniq -c > sorted_bwa_mem_aln.bam.cov.hist
```

```bash
bwa mem extractedGTF_gencode.v33.transcripts.fa $DATA/reps_unitigs_SRR11015356_k75.fa -t 4 > bwa_aln_reps_unitigs_SRR11015356_k75.sam
samtools view -S -b bwa_aln_reps_unitigs_SRR11015356_k75.sam -o bwa_aln_reps_unitigs_SRR11015356_k75.bam
samtools sort bwa_aln_reps_unitigs_SRR11015356_k75.bam -o sorted_bwa_aln_reps_unitigs_SRR11015356_k75.bam
samtools index sorted_bwa_aln_reps_unitigs_SRR11015356_k75.bam
samtools depth sorted_bwa_aln_reps_unitigs_SRR11015356_k75.bam > sorted_bwa_aln_reps_unitigs_SRR11015356_k75.bam.cov
cat sorted_bwa_aln_reps_unitigs_SRR11015356_k75.bam.cov | awk -F'\t' '{print $3}' | sort -n | uniq -c > sorted_bwa_aln_reps_unitigs_SRR11015356_k75.bam.cov.hist
```

```bash
paste sorted_bwa_aln_reps_unitigs_SRR11015356_k75.bam.cov.hist sorted_bwa_mem_aln.bam.cov.hist
```